In [7]:
import sys
sys.path.append('/Users/Jack/data/bvs/python')

In [8]:
import numpy as np, pylab as plt, pandas as pd
import mendeleev
from numpy import ravel_multi_index as rav
from scipy.special import erfc
from BVEL_config import atomicNumbers, atomicSymbols, principleQuantumNumbers, covalentRadii
from BVEL import UnitCell, BVEL_params

In [9]:
c = 'nhs'

In [10]:
alk = 'Na'

In [11]:
#lookup_lanthanide = {'lts':'Tb','lhs':'Ho','lys':'Y','les':'Er'}
lookup_lanthanide = {'nts':'Tb','nhs':'Ho','nys':'Y','nes':'Er'}

In [12]:
cif = '{}_neutron.cif'.format(c.upper())
cutoffRadii = 6.
dx,dy,dz = 0.1,0.1,0.1

In [13]:
ionChargeDict = {lookup_lanthanide[c]:3, alk:1, 'O':-2, 'Si':4,}

unitCell = UnitCell(cif,charges=ionChargeDict)
testIons = unitCell.get_ionOfType(alk)
negativeIons = unitCell.get_ionOfType('O')
positiveIons = unitCell.get_counterCations(alk)

/Users/Jack/code/miniconda3/lib/python3.7/site-packages/ase/io/cif.py:125: UserWarning: Wrong number of tokens: ['?', "#<--'Last", 'name,', 'first', "name'"]
  warnings.warn('Wrong number of tokens: {0}'.format(tokens))


In [14]:
resolution = np.array([dx,dy,dz])
sizeOfValenceMismatchArray = tuple((unitCell.latticeParameters / resolution).astype(int))
BVEL = np.ones(sizeOfValenceMismatchArray)

In [15]:
D0, a, Rmin, R0 = BVEL_params.loc[
    (testIons.label,testIons.charge),
    ['D0','alpha','Rmin','R0']].values

In [16]:
def get_oxidationState(atomicNumber):
    return unitCell.charges[atomicSymbols[atomicNumber]]

In [17]:
testIons.covalentRadii = covalentRadii[testIons.atomicNumber]

In [18]:
numberOfVoxels = len(BVEL.flatten())
for voxelFlatIndex in range(numberOfVoxels):
#for voxelFlatIndex in range(3):    

    voxelTupleIndex = np.unravel_index(voxelFlatIndex,BVEL.shape)
    voxelScaledPosition = np.array(voxelTupleIndex)/np.array(BVEL.shape)
    voxelUnscaledPosition = np.dot(unitCell.latticeVectors.T,voxelScaledPosition)
    
    # ---------- Attractive Term -----------------
    negativeIonsInCutoff = negativeIons.get_ionsInCutoff(position=voxelUnscaledPosition,
                                                         cutoff=cutoffRadii)
    
    distanceToNegativeIons = np.linalg.norm(negativeIonsInCutoff.get_positions() \
                                                   - voxelUnscaledPosition, axis=1)
    Ri = distanceToNegativeIons
    #print(len(Ri))
    Ri = Ri[Ri<cutoffRadii]
    #print((len(Ri)))
    
    Ui = D0 * ( (np.exp(a*(Rmin - Ri)) - 1 )**2 - 1 )
    attractiveTerm = np.sum(Ui)
    
    # ---------- Repulsive Term -----------------
    positiveIonsInCutoff = positiveIons.get_ionsInCutoff(position=voxelUnscaledPosition,
                                                         cutoff=cutoffRadii)
    
    positiveIonsInCutoff.distance = np.linalg.norm(positiveIonsInCutoff.get_positions() \
                                               -voxelUnscaledPosition, axis=1)
    Rj = positiveIonsInCutoff.distance
    cutIndex = Rj<cutoffRadii
    Rj = Rj[cutIndex]
    #print(len(Rj))
    
    positiveIonsInCutoff.distance = positiveIonsInCutoff.distance[cutIndex]
    
    positiveIonsInCutoff.covalentRadii = np.array([covalentRadii[atomicNumber] \
                                        for atomicNumber in positiveIonsInCutoff.numbers])[cutIndex]

    positiveIonsInCutoff.principleQuantumNumbers = np.array([principleQuantumNumbers[atomicNumber]\
                                        for atomicNumber in positiveIonsInCutoff.numbers])[cutIndex]
    
    positiveIonsInCutoff.oxidationStates = np.array([get_oxidationState(atomicNumber) \
                                        for atomicNumber in positiveIonsInCutoff.numbers])[cutIndex]
    
    C = 14.40198 # conversion factor = (boltzmann's constant) * (eV/J) * (meters/Angstrom)
    d = positiveIonsInCutoff.distance
    q = positiveIonsInCutoff.oxidationStates
    n = positiveIonsInCutoff.principleQuantumNumbers
    r = positiveIonsInCutoff.covalentRadii
    p = 0.74 * (testIons.covalentRadii + r)
    
    Uj = C*q/(2*np.sqrt(2*n)) * (erfc(d/p) - erfc(cutoffRadii/p))
    repulsiveTerm = np.sum( Uj )

    # ------------- sum it up ---------------------
    
    BVEL[voxelTupleIndex] = attractiveTerm + repulsiveTerm 
    

In [19]:
import os
if not os.path.isdir('./grds/'):
    os.mkdir('./grds/')

In [20]:
cif

'NHS_neutron.cif'

In [21]:
BVEL = BVEL - BVEL.min()
#output_filename = 'myBVEL.grd'

#output_filename = 'grds/'+cif.split('.')[0].split('/')[1] + '.grd'
output_filename = 'grds/'+cif.split('.')[0] + '_highRes.grd'

with open(output_filename,"w") as savefile:
    savefile.write("Bond Valence Sum Difference\r") # Title
    from ase.geometry import cell_to_cellpar
    cellParams = cell_to_cellpar(unitCell.latticeVectors) # get ABC alpha, beta, gamma
    savefile.write(" ".join([str(k) for k in cellParams])+"\r" )
    savefile.write(" ".join([str(k) for k in BVEL.shape])+"\r" )
    for i in np.nditer(BVEL.flatten()):
        savefile.write("%.6f  "%(i)) # Write each valence difference value